<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames,CSV,JWAS,JWAS.Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [4]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [5]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [6]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [9]:
model    = build_model(model_equations,R);

In [10]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [11]:
ped = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [12]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [14]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                         10
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)...  0%|      |  ETA: 0:07:55


Posterior means at iteration: 500

running MCMC for conventional (no markers)... 25%|██    |  ETA: 0:00:02


Residual covariance matrix: 
[38.2784 8.53017; 8.53017 2.54553]
Polygenic effects covariance matrix 
[120.265 26.6641; 26.6641 8.25117]



running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:01


Posterior means at iteration: 1000
Residual covariance matrix: 
[40.7927 9.60445; 9.60445 2.88894]
Polygenic effects covariance matrix 
[109.396 23.709; 23.709 7.38335]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [15]:
out["MCMC samples for residual covariance matrix"]

4×100 Array{Float64,2}:
 140.286    165.117   27.4022   47.8215   …  100.707    114.361    87.6562 
  19.8049    48.1317   6.39921   9.76642      27.2255    23.6785   22.9175 
  19.8049    48.1317   6.39921   9.76642      27.2255    23.6785   22.9175 
   3.25893   14.6208   2.0417    2.35088       7.86628    5.32089   6.31246

In [16]:
using JWAS:misc
reformat(out["MCMC samples for residual covariance matrix"])

100-element Array{Array{Float64,2},1}:
 [140.286 19.8049; 19.8049 3.25893]
 [165.117 48.1317; 48.1317 14.6208]
 [27.4022 6.39921; 6.39921 2.0417] 
 [47.8215 9.76642; 9.76642 2.35088]
 [32.3853 6.17495; 6.17495 1.50397]
 [133.801 37.3524; 37.3524 10.6991]
 [54.6587 12.1139; 12.1139 2.97054]
 [147.844 48.5646; 48.5646 16.3886]
 [92.846 27.3524; 27.3524 8.32298] 
 [92.0702 24.6955; 24.6955 6.7504] 
 [266.548 75.1539; 75.1539 21.5001]
 [162.668 35.0212; 35.0212 7.80687]
 [26.0195 9.02234; 9.02234 3.36369]
 ⋮                                 
 [90.9382 19.8054; 19.8054 4.45033]
 [66.0173 17.1109; 17.1109 4.62523]
 [93.3314 28.7019; 28.7019 9.23725]
 [82.5146 24.6105; 24.6105 7.65399]
 [129.921 36.5742; 36.5742 10.4571]
 [69.667 18.2625; 18.2625 5.10302] 
 [84.0481 18.6961; 18.6961 4.36653]
 [145.568 41.1245; 41.1245 12.0517]
 [39.3698 13.6979; 13.6979 4.99284]
 [100.707 27.2255; 27.2255 7.86628]
 [114.361 23.6785; 23.6785 5.32089]
 [87.6562 22.9175; 22.9175 6.31246]

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [17]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [18]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [19]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [20]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

INFO: dam is not found in model equation 2.


In [21]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)... 23%|█     |  ETA: 0:00:00


Posterior means at iteration: 500
Residual covariance matrix: 
[127.85 29.8952; 29.8952 7.57724]
Polygenic effects covariance matrix 
[6.51745 1.40374 0.121162; 1.40374 1.02113 -0.007159; 0.121162 -0.007159 0.416418]



running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:00


Posterior means at iteration: 1000
Residual covariance matrix: 
[133.3 31.3482; 31.3482 7.94058]
Polygenic effects covariance matrix 
[5.91959 1.24817 0.140792; 1.24817 0.984565 0.035887; 0.140792 0.035887 0.43876]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00


In [22]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of location parameters"